In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
MP = os.path.join(os.pardir, 'metadata')

In [3]:
def get_columname(PATH):
    for i in list(os.listdir(PATH)):
        file = os.path.join(PATH, i)
        instance = np.load(file, allow_pickle=True)
        column = []
        column += ['z_mu_' + str(j) for j in range(len(instance['z_mu']))]
        column += ['z_sig_' + str(j) for j in range(len(instance['z_sig']))]
        column += ['vp' + str(j) for j in range(len(instance['vp']))]
        column.append('y')
        return column

def load_meta(PATH):
    total = []
    for i in list(os.listdir(PATH)):
        instance_list= []
        file = os.path.join(PATH, i)
        instance = np.load(file, allow_pickle=True)
        instance_list.extend(instance['z_mu'])
        instance_list.extend(instance['z_sig'])
        instance_list.extend(instance['vp'])
        instance_list.append(int(instance['y'].item()))
        total.append(instance_list)
    
    total = np.array(total)
    df = pd.DataFrame(data = total, columns=get_columname(PATH))
    df = df.astype({'vp0':int, 'vp1':int, 'vp2':int, 'y':int})
    return df
        

In [4]:
a = load_meta(MP)

In [5]:
a

,z_mu_0,z_mu_1,z_mu_2,z_mu_3,z_mu_4,z_mu_5,z_mu_6,z_mu_7,z_mu_8,z_mu_9,...,z_sig_244,z_sig_245,z_sig_246,z_sig_247,z_sig_248,z_sig_249,vp0,vp1,vp2,y
0,6.776926e-17,1.0,1.0,0.0,0.016626,7.650776e-19,1.0,8.569905e-18,0.0,1.200030e-29,...,1.0,2.718282,1.0,2.718282,2.718282,1.0,0,0,1,0
1,6.776771e-17,1.0,1.0,0.0,0.016626,7.650659e-19,1.0,8.569905e-18,0.0,1.200030e-29,...,1.0,2.718282,1.0,2.718282,2.718282,1.0,0,0,1,10
2,6.776926e-17,1.0,1.0,0.0,0.016626,7.650893e-19,1.0,8.569905e-18,0.0,1.200030e-29,...,1.0,2.718282,1.0,2.718282,2.718282,1.0,0,0,1,130
3,6.776771e-17,1.0,1.0,0.0,0.016626,7.650688e-19,1.0,8.569905e-18,0.0,1.200030e-29,...,1.0,2.718282,1.0,2.718282,2.718282,1.0,0,0,1,72
4,6.776771e-17,1.0,1.0,0.0,0.016626,7.650513e-19,1.0,8.569905e-18,0.0,1.200030e-29,...,1.0,2.718282,1.0,2.718282,2.718282,1.0,0,0,1,413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5897,6.776926e-17,1.0,1.0,0.0,0.016626,7.650688e-19,1.0,8.569905e-18,0.0,1.200030e-29,...,1.0,2.718282,1.0,2.718282,2.718282,1.0,0,0,1,75
5898,6.776926e-17,1.0,1.0,0.0,0.016626,7.650572e-19,1.0,8.569905e-18,0.0,1.200030e-29,...,1.0,2.718282,1.0,2.718282,2.718282,1.0,0,0,1,108
5899,6.776926e-17,1.0,1.0,0.0,0.016626,7.650922e-19,1.0,8.569905e-18,0.0,1.200030e-29,...,1.0,2.718282,1.0,2.718282,2.718282,1.0,0,0,1,354
5900,6.782745e-17,1.0,1.0,0.0,0.016626,7.651009e-19,1.0,8.565526e-18,0.0,1.201212e-29,...,1.0,2.718282,1.0,2.718282,2.718282,1.0,1,0,0,83


## XGBOOST

In [6]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [15]:
b = a.y.value_counts()

In [24]:
b.keys()

Int64Index([ 93,  15,  66,  10,  62,  36,  82, 189,  28,  77,
            ...
            671, 677, 678, 682, 207, 683, 685, 209, 687, 400],
           dtype='int64', length=788)

In [25]:
nl = [k for k in list(b.keys()) if b[k]>=6]

In [57]:
ndf = a[a['y'].isin(nl)]
ndf_2 = ndf.replace({'y':{item:i for i, item in enumerate(np.unique(ndf.y.values))}})

In [62]:
#X, y = a.iloc[:,:-1].values, a.iloc[:,-1].values
X, y = ndf_2.iloc[:,:-1].values, ndf_2.iloc[:,-1].values


In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [64]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='mlogloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', use_label_encoder=False,
              validate_parameters=1, ...)

In [65]:
y_pred = model.predict(X_test)

In [66]:
accuracy_score(y_test, y_pred)

0.0021141649048625794

## NN

In [85]:
a.shape

(5902, 504)

In [79]:
lol = sorted(list(a.y.unique()))
print(len(lol))

788


In [80]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm



class CustomTrain(nn.Module):
    def __init__(self, n_features, n_classes, layers_list, activation=nn.ReLU(), dropout_list=None, batch_norm=True):
        super(CustomTrain, self).__init__()
        self.n_features = n_features
        self.n_classes = n_classes
        self.layers_list = layers_list
        self.activation = activation
        self.dropout_list = dropout_list
        self.batch_norm = batch_norm
        self.net = []
        self.b_list = []
        if self.dropout_list:
            self.dropout_list = [nn.Dropout(i) for i in self.dropout_list]
        for i in range(len(self.layers_list)):
            if self.batch_norm:
                self.b_list.append(nn.BatchNorm1d(self.layers_list[i]))
            if i==0:
                self.net.append(nn.Linear(self.n_features, self.layers_list[i]))
            else:
                self.net.append(nn.Linear(self.layers_list[i-1], self.layers_list[i]))
        self.last_layer = nn.Linear(self.layers_list[-1], self.n_classes)
        self.net = nn.ModuleList(self.net)
        
    
    def forward(self, x):

        for i, l in enumerate(self.net):
            #print(i)
            x = self.activation(l(x))
            if self.batch_norm:
                x = self.b_list[i](x)
            if self.dropout_list:
                x = (self.dropout_list[i])(x)
            
        x = self.last_layer(x)
        if self.n_classes == 1:
            x = torch.sigmoid(x)
   
        return x

class datapaltas(Dataset):
    def __init__(self, df, scale =True, y_idx=-1):
        self.df = df
        self.y_idx = y_idx

    def __getitem__(self, index):
        X = (self.df.iloc[index,:self.y_idx]).values
        X = X.astype(np.float64)
        X = torch.from_numpy(X).float()
        y = self.df.iloc[index,self.y_idx]
        #print(y)
        y = torch.Tensor([y]).long()
        return {'x':X, 'y':y}
    
    def __len__(self):
        return self.df.shape[0]
    
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc


In [81]:
ds = datapaltas(a)

In [86]:
device = 'cpu'
def train(data, model, ep = 120, save=False, prefix=None):
    idx_result = {}
    a = datapaltas(df = data, y_idx=-1)
    EPOCHS = ep
    skf = StratifiedKFold()
    ## y_idx -2
    skf.get_n_splits(data, data.iloc[:,-1])
    vec_train = np.array([train_ids for train_ids,_ in skf.split(data, data.iloc[:,-1])])
    vec_test = np.array([test_ids for _,test_ids in skf.split(data, data.iloc[:,-1])])
    acv, tav = [], []
    foldn = 0
    for train_ids, test_ids in zip(vec_train, vec_test):
        acv_, tav_ = [], []
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
        train_loader = torch.utils.data.DataLoader(
                            a, 
                            batch_size=64, sampler=train_subsampler)
        test_loader = torch.utils.data.DataLoader(
                            a,
                            batch_size=1, sampler=test_subsampler)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr = 0.001)
        EPOCHS = ep
        itt = tqdm(range(EPOCHS))
        for i in itt:
            loss_epoch = 0
            acc_train, acc_test = 0.0, 0.0
            model.train()
            for sample in train_loader:
                X, y = sample['x'], sample['y']
                X, y = X.to(device), y.to(device).flatten()
                optimizer.zero_grad()
                y_pred = model(X)
                #print(y_pred, y)
                loss = criterion(y_pred, y)
                acc = multi_acc(y_pred, y)
                loss.backward()
                optimizer.step()
                loss_epoch += loss.item()
                acc_train += acc.item()
                
            with torch.no_grad():

                for sample in test_loader:
                    X, y = sample['x'], sample['y']
                    X, y = X.to(device), y.to(device).flatten()
                    model.eval()
                    output = model(X)
                    acc = multi_acc(output, y)
                    acc_test += acc.item()
                
            acv_.append(acc_test/len(test_loader))
            tav_.append(acc_train/len(train_loader))
            #print(acv_)
            if save and (abs(acc_test/len(test_loader) - max(acv_)<1e-8)):
                    if os.path.exists(os.path.join(os.curdir, prefix+"model_fold_"+str(foldn)+".pth")):
                        os.remove(os.path.join(os.curdir, prefix+"model_fold_"+str(foldn)+".pth"))
                    torch.save(model, os.path.join(os.curdir, prefix+"model_fold_"+str(foldn)+".pth"))
                    itt.set_postfix({'epoch_model': i, 'best_acc':max(acv_)})
            t1, t2 = acc_train/len(train_loader), acc_test/len(test_loader)
            itt.set_description(f"Acc train: {t1:.2f} Acc test: {t2:.2f}")
        acv.append(acv_)
        tav.append(tav_)
        

        foldn += 1   
        
    acv = np.array(acv)
    tav = np.array(tav)
    idx_result['acc'] = np.mean(acv, axis = 0)
    idx_result['acc_std'] = np.std(acv, axis=0)
    idx_result['train_acc'] = np.mean(tav, axis=0)
    return idx_result



In [87]:

model = CustomTrain(503, 788, [128, 64, 32, 16], nn.SiLU(), batch_norm=False)
model

CustomTrain(
  (activation): SiLU()
  (last_layer): Linear(in_features=16, out_features=788, bias=True)
  (net): ModuleList(
    (0): Linear(in_features=503, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=16, bias=True)
  )
)

In [89]:
eo = train(a, model, ep=200)

C:\Users\abdig\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,
C:\Users\abdig\miniconda3\envs\sale\lib\site-packages\ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':
C:\Users\abdig\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,
C:\Users\abdig\miniconda3\envs\sale\lib\site-packages\ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or nd

KeyboardInterrupt: 